In [101]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [102]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [103]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [104]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [105]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [106]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [107]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [108]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [109]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [110]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [111]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [112]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [113]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Nov-20,"25,334","24,482",852,3.4,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Nov-20,"92,160","88,745","3,415",3.7,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Nov-20,"9,733","9,190",543,5.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Nov-20,"8,627","8,251",376,4.4,01007
38631,CN0100900000000,1,9,"Blount County, AL",Nov-20,"24,849","24,210",639,2.6,01009


In [114]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [115]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Nov-20,"12,447","10,911",1536,12.3,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Nov-20,"2,445","2,166",279,11.4,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Nov-20,"6,613","5,859",754,11.4,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Nov-20,"7,907","7,027",880,11.1,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Nov-20,"9,202","8,089",1113,12.1,72153


In [116]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [117]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [118]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [119]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [120]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [121]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [122]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [123]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [124]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1066387,2021-02-25,Webster,West Virginia,54101.0,308,2.0
1066388,2021-02-25,Wetzel,West Virginia,54103.0,1075,19.0
1066389,2021-02-25,Wirt,West Virginia,54105.0,351,2.0
1066390,2021-02-25,Wood,West Virginia,54107.0,7002,130.0
1066391,2021-02-25,Wyoming,West Virginia,54109.0,1736,34.0
1066392,2021-02-25,Adams,Wisconsin,55001.0,1736,15.0
1066393,2021-02-25,Ashland,Wisconsin,55003.0,1238,16.0
1066394,2021-02-25,Barron,Wisconsin,55005.0,5474,76.0
1066395,2021-02-25,Bayfield,Wisconsin,55007.0,1136,19.0
1066396,2021-02-25,Brown,Wisconsin,55009.0,32538,249.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [125]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
355,02/18/2021,1921,852,242,48,9,2703,3642,280,72,78,355,157,54,7,0,553,745,71,15,15,566,262,74,17,1,797,1074,82,23,24,377,166,37,7,1,412,562,41,9,10,520,225,69,15,1,771,1033,73,21,22,103,42,8,2,0,169,227,13,4,4,11000
356,02/19/2021,2868,1029,286,53,6,2625,3583,273,69,75,588,203,83,9,0,530,722,72,14,15,816,329,72,15,1,777,1064,78,22,23,443,158,38,11,0,401,553,40,10,10,802,267,82,18,0,749,1014,72,20,21,217,72,11,0,1,166,228,11,3,3,11000
357,02/20/2021,2318,964,276,47,19,2643,3612,274,65,72,476,202,82,17,1,539,734,73,15,15,636,285,70,11,0,775,1067,78,20,21,324,157,39,6,0,409,561,40,9,9,726,269,75,7,1,751,1020,72,18,18,156,51,10,6,0,169,230,11,3,3,11000
358,02/21/2021,2072,553,174,41,11,2679,3599,265,60,69,323,83,59,15,1,534,715,72,15,16,722,184,32,10,0,798,1078,73,18,19,278,64,19,5,0,414,556,38,8,8,606,188,58,9,0,761,1019,72,17,17,143,34,6,2,0,173,230,10,3,3,11000
359,02/22/2021,2489,1025,48,38,19,2643,3595,228,55,65,540,176,39,8,0,539,725,67,14,15,766,322,4,13,0,790,1083,61,16,17,351,171,2,7,0,413,565,32,7,7,665,287,3,7,1,736,999,60,15,15,166,69,0,3,0,165,223,8,3,3,11000


In [126]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [127]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [128]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [129]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [130]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [131]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [132]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
355,02/18/2021,355,7,New York,Bronx,36005.0
356,02/19/2021,588,9,New York,Bronx,36005.0
357,02/20/2021,476,17,New York,Bronx,36005.0
358,02/21/2021,323,15,New York,Bronx,36005.0
359,02/22/2021,540,8,New York,Bronx,36005.0


In [133]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [134]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,1,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [135]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [136]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
355,02/18/2021,566,17,Kings,New York,36047.0
356,02/19/2021,816,15,Kings,New York,36047.0
357,02/20/2021,636,11,Kings,New York,36047.0
358,02/21/2021,722,10,Kings,New York,36047.0
359,02/22/2021,766,13,Kings,New York,36047.0


In [137]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [138]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [139]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [140]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,1,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [141]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,8521,209


In [142]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [143]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [144]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
346,02/09/2021,972,21,New York,Queens,36081.0,2021-02-09
347,02/10/2021,992,23,New York,Queens,36081.0,2021-02-10
348,02/11/2021,958,25,New York,Queens,36081.0,2021-02-11
349,02/12/2021,956,26,New York,Queens,36081.0,2021-02-12
350,02/13/2021,711,24,New York,Queens,36081.0,2021-02-13
351,02/14/2021,539,16,New York,Queens,36081.0,2021-02-14
352,02/15/2021,840,21,New York,Queens,36081.0,2021-02-15
353,02/16/2021,973,26,New York,Queens,36081.0,2021-02-16
354,02/17/2021,859,21,New York,Queens,36081.0,2021-02-17
355,02/18/2021,520,15,New York,Queens,36081.0,2021-02-18


In [145]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
310,01/04/2021,Richmond,New York,36085.0,655,8.0,2021-01-04
311,01/05/2021,Richmond,New York,36085.0,515,5.0,2021-01-05
312,01/06/2021,Richmond,New York,36085.0,524,6.0,2021-01-06
313,01/07/2021,Richmond,New York,36085.0,453,7.0,2021-01-07
314,01/08/2021,Richmond,New York,36085.0,480,8.0,2021-01-08
315,01/09/2021,Richmond,New York,36085.0,371,5.0,2021-01-09
316,01/10/2021,Richmond,New York,36085.0,342,4.0,2021-01-10
317,01/11/2021,Richmond,New York,36085.0,466,2.0,2021-01-11
318,01/12/2021,Richmond,New York,36085.0,415,5.0,2021-01-12
319,01/13/2021,Richmond,New York,36085.0,379,6.0,2021-01-13


In [146]:
print (data_date_dt)

2021-02-26 00:00:00


In [147]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1063242,1001.0,89.0
1063243,1003.0,274.0


In [148]:
Kings_death.head(2)

,fips,deaths
0,36047.0,7243


In [149]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1066242,51690.0,51.0
1066243,51115.0,9.0
1066244,51117.0,50.0
1066245,51119.0,16.0
1066246,51121.0,74.0
1066247,51125.0,7.0
1066248,51127.0,9.0
1066249,51700.0,141.0
1066250,51710.0,175.0
1066251,51131.0,32.0


In [150]:
covid_death.shape

(3250, 2)

In [151]:
print (County_covid_death)

{1001.0: 89.0, 1003.0: 274.0, 1005.0: 51.0, 1007.0: 60.0, 1009.0: 125.0, 1011.0: 36.0, 1013.0: 65.0, 1015.0: 285.0, 1017.0: 108.0, 1019.0: 38.0, 1021.0: 98.0, 1023.0: 23.0, 1025.0: 49.0, 1027.0: 54.0, 1029.0: 39.0, 1031.0: 102.0, 1033.0: 118.0, 1035.0: 23.0, 1037.0: 23.0, 1039.0: 106.0, 1041.0: 54.0, 1043.0: 179.0, 1045.0: 106.0, 1047.0: 141.0, 1049.0: 174.0, 1051.0: 183.0, 1053.0: 72.0, 1055.0: 319.0, 1057.0: 56.0, 1059.0: 76.0, 1061.0: 70.0, 1063.0: 32.0, 1065.0: 68.0, 1067.0: 41.0, 1069.0: 259.0, 1071.0: 102.0, 1073.0: 1356.0, 1075.0: 32.0, 1077.0: 210.0, 1079.0: 84.0, 1081.0: 150.0, 1083.0: 134.0, 1085.0: 48.0, 1087.0: 43.0, 1089.0: 442.0, 1091.0: 54.0, 1093.0: 94.0, 1095.0: 206.0, 1097.0: 721.0, 1099.0: 38.0, 1101.0: 494.0, 1103.0: 249.0, 1105.0: 27.0, 1107.0: 54.0, 1109.0: 71.0, 1111.0: 40.0, 1113.0: 31.0, 1117.0: 215.0, 1115.0: 220.0, 1119.0: 31.0, 1121.0: 163.0, 1123.0: 141.0, 1125.0: 409.0, 1127.0: 251.0, 1129.0: 34.0, 1131.0: 25.0, 1133.0: 66.0, 2013.0: 1.0, 2016.0: 0.0, 2020

In [152]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [153]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [154]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1021360,2021-02-12,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-12,-14 days,NaN
1024605,2021-02-13,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-13,-13 days,0.0
1027850,2021-02-14,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-14,-12 days,0.0
1031097,2021-02-15,New Haven,Connecticut,9009.0,68437,1874.0,2021-02-15,-11 days,851.0
1034343,2021-02-16,New Haven,Connecticut,9009.0,68593,1873.0,2021-02-16,-10 days,156.0
1037588,2021-02-17,New Haven,Connecticut,9009.0,68795,1880.0,2021-02-17,-9 days,202.0
1040833,2021-02-18,New Haven,Connecticut,9009.0,68953,1885.0,2021-02-18,-8 days,158.0
1044078,2021-02-19,New Haven,Connecticut,9009.0,69233,1891.0,2021-02-19,-7 days,280.0
1047323,2021-02-20,New Haven,Connecticut,9009.0,69233,1891.0,2021-02-20,-6 days,0.0
1050570,2021-02-21,New Haven,Connecticut,9009.0,69233,1891.0,2021-02-21,-5 days,0.0


In [155]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [156]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,3307.0


In [157]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
310,01/04/2021,Richmond,New York,36085.0,655,8.0,2021-01-04
311,01/05/2021,Richmond,New York,36085.0,515,5.0,2021-01-05
312,01/06/2021,Richmond,New York,36085.0,524,6.0,2021-01-06
313,01/07/2021,Richmond,New York,36085.0,453,7.0,2021-01-07
314,01/08/2021,Richmond,New York,36085.0,480,8.0,2021-01-08
315,01/09/2021,Richmond,New York,36085.0,371,5.0,2021-01-09
316,01/10/2021,Richmond,New York,36085.0,342,4.0,2021-01-10
317,01/11/2021,Richmond,New York,36085.0,466,2.0,2021-01-11
318,01/12/2021,Richmond,New York,36085.0,415,5.0,2021-01-12
319,01/13/2021,Richmond,New York,36085.0,379,6.0,2021-01-13


In [158]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [159]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [160]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [161]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [162]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,12.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [163]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-02-26 15:11:14.819302
2


In [164]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [165]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [166]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [167]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [168]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 89.0
1003.0 274.0
1005.0 51.0
1007.0 60.0
1009.0 125.0
1011.0 36.0
1013.0 65.0
1015.0 285.0
1017.0 108.0
1019.0 38.0
1021.0 98.0
1023.0 23.0
1025.0 49.0
1027.0 54.0
1029.0 39.0
1031.0 102.0
1033.0 118.0
1035.0 23.0
1037.0 23.0
1039.0 106.0
1041.0 54.0
1043.0 179.0
1045.0 106.0
1047.0 141.0
1049.0 174.0
1051.0 183.0
1053.0 72.0
1055.0 319.0
1057.0 56.0
1059.0 76.0
1061.0 70.0
1063.0 32.0
1065.0 68.0
1067.0 41.0
1069.0 259.0
1071.0 102.0
1073.0 1356.0
1075.0 32.0
1077.0 210.0
1079.0 84.0
1081.0 150.0
1083.0 134.0
1085.0 48.0
1087.0 43.0
1089.0 442.0
1091.0 54.0
1093.0 94.0
1095.0 206.0
1097.0 721.0
1099.0 38.0
1101.0 494.0
1103.0 249.0
1105.0 27.0
1107.0 54.0
1109.0 71.0
1111.0 40.0
1113.0 31.0
1117.0 215.0
1115.0 220.0
1119.0 31.0
1121.0 163.0
1123.0 141.0
1125.0 409.0
1127.0 251.0
1129.0 34.0
1131.0 25.0
1133.0 66.0
2013.0 1.0
2016.0 0.0
2020.0 152.0
2050.0 18.0
2997.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 26.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 2.0
2150.0 5.0
2158.0 3.0
2170.0

23009.0 34.0
23011.0 51.0
23013.0 5.0
23015.0 2.0
23017.0 50.0
23019.0 85.0
23021.0 1.0
23023.0 5.0
23025.0 27.0
23027.0 19.0
23029.0 14.0
23031.0 116.0
24001.0 198.0
24003.0 532.0
24005.0 1275.0
24510.0 885.0
24009.0 69.0
24011.0 19.0
24013.0 212.0
24015.0 119.0
24017.0 157.0
24019.0 45.0
24021.0 272.0
24023.0 61.0
24025.0 236.0
24027.0 219.0
24029.0 42.0
24031.0 1408.0
24033.0 1327.0
24035.0 38.0
24039.0 31.0
24037.0 114.0
24041.0 33.0
nan 28.0
24043.0 252.0
24045.0 141.0
24047.0 92.0
25001.0 398.0
25003.0 254.0
25005.0 1482.0
25007.0 0.0
25009.0 2141.0
25011.0 100.0
25013.0 1320.0
25015.0 258.0
25017.0 3416.0
25019.0 2.0
25021.0 1607.0
25023.0 1283.0
25025.0 1685.0
nan 8.0
25027.0 2024.0
26001.0 24.0
26003.0 5.0
26005.0 118.0
26007.0 47.0
26009.0 20.0
26011.0 24.0
26013.0 33.0
26015.0 44.0
26017.0 287.0
26019.0 31.0
26021.0 239.0
26023.0 110.0
26025.0 234.0
26027.0 64.0
26029.0 13.0
26031.0 35.0
26033.0 43.0
26035.0 69.0
26037.0 70.0
26039.0 5.0
26041.0 84.0
26043.0 69.0
26045.0 152

48031.0 17.0
48033.0 2.0
48035.0 30.0
48037.0 179.0
48039.0 371.0
48041.0 213.0
48043.0 10.0
48045.0 4.0
48047.0 32.0
48049.0 125.0
48051.0 34.0
48053.0 58.0
48055.0 82.0
48057.0 16.0
48059.0 37.0
48061.0 1466.0
48063.0 40.0
48065.0 14.0
48067.0 76.0
48069.0 29.0
48071.0 42.0
48073.0 120.0
48075.0 16.0
48077.0 16.0
48079.0 13.0
48081.0 11.0
48083.0 28.0
48085.0 712.0
48087.0 9.0
48089.0 24.0
48091.0 284.0
48093.0 50.0
48095.0 7.0
48097.0 62.0
48099.0 72.0
48101.0 7.0
48103.0 11.0
48105.0 15.0
48107.0 26.0
48109.0 7.0
48111.0 22.0
48113.0 3356.0
48115.0 68.0
48123.0 56.0
48117.0 63.0
48119.0 10.0
48121.0 634.0
48125.0 9.0
48127.0 19.0
48129.0 13.0
48131.0 37.0
48133.0 42.0
48135.0 322.0
48137.0 4.0
48141.0 2322.0
48139.0 272.0
48143.0 65.0
48145.0 29.0
48147.0 99.0
48149.0 52.0
48151.0 13.0
48153.0 29.0
48155.0 8.0
48157.0 545.0
48159.0 24.0
48161.0 42.0
48163.0 41.0
48165.0 40.0
48167.0 336.0
48169.0 19.0
48171.0 47.0
48173.0 2.0
48175.0 13.0
48177.0 50.0
48179.0 53.0
48181.0 308.0
481

In [169]:
K_c.head()

,fips,cases,deaths
0,36047.0,8521,209


In [170]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [171]:
print (K)

{36047.0: 8521}


In [172]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 65.0
1001 219.0
1009 133.0
1013 59.0
1015 598.0
1017 80.0
1031 95.0
1033 65.0
1039 142.0
1043 88.0
1045 141.0
1051 277.0
1055 229.0
1067 51.0
1069 206.0
1071 81.0
1077 139.0
1079 51.0
1083 216.0
1089 943.0
1095 158.0
1097 1249.0
1103 253.0
1111 60.0
1113 101.0
1115 228.0
1117 656.0
1121 243.0
2261 22.0
4021 1435.0
5009 87.0
5011 10.0
5033 118.0
5037 27.0
5045 257.0
5047 53.0
5051 413.0
5053 40.0
5063 49.0
5083 54.0
5085 218.0
5087 16.0
5115 197.0
5117 10.0
5121 29.0
5125 236.0
5131 323.0
5145 148.0
6007 291.0
6017 295.0
6023 151.0
6027 110.0
6061 538.0
6079 723.0
6089 232.0
6093 76.0
8019 5.0
8039 49.0
8047 7.0
8051 62.0
8057 3.0
8065 40.0
8067 106.0
8071 30.0
8075 29.0
8077 325.0
8083 34.0
8085 90.0
8093 22.0
8099 8.0
8101 218.0
8103 29.0
8119 40.0
8121 17.0
9005 478.0
9007 468.0
9011 870.0
9015 279.0
10001 625.0
12019 512.0
12033 884.0
12089 275.0
12101 1664.0
12109 581.0
12113 582.0
13015 796.0
13039 187.0
13045 346.0
13047 302.0
13057 1539.0
13073 624.0
13077 681.0
13085 103.

48059 27.0
48065 14.0
48067 54.0
48077 7.0
48083 6.0
48091 203.0
48097 84.0
48119 9.0
48143 66.0
48149 50.0
48151 0.0
48159 23.0
48175 0.0
48179 123.0
48181 391.0
48183 189.0
48187 768.0
48203 129.0
48221 153.0
48223 83.0
48233 25.0
48235 9.0
48241 67.0
48251 548.0
48259 264.0
48277 190.0
48281 66.0
48307 18.0
48337 36.0
48343 27.0
48361 448.0
48363 55.0
48367 484.0
48373 54.0
48381 134.0
48387 14.0
48395 38.0
48399 9.0
48441 200.0
48459 55.0
48467 115.0
48477 74.0
48503 52.0
49017 12.0
49025 68.0
49055 3.0
50003 141.0
50005 28.0
50011 194.0
50013 14.0
50015 58.0
50017 38.0
50019 24.0
50023 97.0
50025 96.0
50027 68.0
51005 100.0
51007 39.0
51009 73.0
51011 46.0
51015 131.0
51017 3.0
51019 201.0
51023 203.0
51031 171.0
51033 58.0
51035 60.0
51041 1611.0
51043 27.0
51045 10.0
51047 99.0
51049 26.0
51053 103.0
51057 24.0
51061 172.0
51063 29.0
51065 77.0
51067 109.0
51069 220.0
51071 68.0
51073 129.0
51075 57.0
51077 36.0
51079 65.0
51083 177.0
51085 340.0
51089 155.0
51093 184.0
51095 16

37143 51.0
37149 75.0
37173 44.0
37175 105.0
37179 977.0
37183 3077.0
37191 329.0
37193 200.0
37195 397.0
37199 57.0
38013 3.0
38015 184.0
38017 257.0
38019 0.0
38023 10.0
38029 2.0
38035 94.0
38037 1.0
38039 1.0
38045 1.0
38047 0.0
38051 0.0
38063 6.0
38071 15.0
38077 27.0
38083 1.0
38087 1.0
38089 55.0
38093 17.0
38101 114.0
38103 15.0
39001 52.0
39031 79.0
39041 578.0
39047 79.0
39049 2762.0
39079 56.0
39087 166.0
39097 88.0
39101 137.0
39115 68.0
39129 169.0
39131 74.0
39173 249.0
40003 14.0
40007 37.0
40009 31.0
40011 20.0
40025 48.0
40029 11.0
40039 61.0
40041 99.0
40043 10.0
40057 8.0
40063 30.0
40069 47.0
40073 49.0
40077 19.0
40079 153.0
40097 96.0
40099 53.0
40101 137.0
40109 1845.0
40111 89.0
40115 76.0
40121 68.0
40127 25.0
40143 2424.0
40151 14.0
41001 37.0
41003 167.0
41005 292.0
41013 39.0
41015 49.0
41019 312.0
41025 30.0
41027 18.0
41031 73.0
41039 417.0
41041 14.0
41057 21.0
41063 9.0
41067 513.0
42011 1230.0
42021 350.0
42023 10.0
42043 772.0
42049 727.0
42059 125.0


48329 96.0
48345 2.0
48409 88.0
48413 8.0
48439 6170.0
48443 0.0
48449 72.0
48457 37.0
48471 280.0
48473 149.0
48495 13.0
49039 107.0
51029 80.0
51081 56.0
51153 1473.0
51590 216.0
51620 54.0
51660 122.0
51683 77.0
51685 19.0
51690 45.0
51700 750.0
51740 453.0
51760 853.0
53025 269.0
54013 7.0
54109 44.0
1065 72.0
2013 9.0
2185 14.0
4009 77.0
4012 46.0
5035 77.0
5095 22.0
5099 22.0
6039 406.0
6071 4385.0
6075 985.0
6077 1600.0
6099 1933.0
12013 28.0
12077 16.0
12079 71.0
12086 15825.0
12093 140.0
12123 53.0
12125 36.0
13019 71.0
13063 1083.0
13095 162.0
13099 24.0
13109 24.0
13215 542.0
13261 48.0
13267 56.0
13301 11.0
13303 48.0
16065 218.0
20057 46.0
21013 153.0
21045 32.0
21065 40.0
21075 13.0
21109 29.0
21121 132.0
21129 25.0
21203 55.0
24510 888.0
26085 5.0
28013 34.0
29223 9.0
30035 10.0
31043 29.0
31047 44.0
34011 542.0
34017 3576.0
34031 2207.0
34039 2455.0
35005 121.0
35019 13.0
35023 15.0
35033 11.0
35045 218.0
35051 21.0
35053 26.0
36047 875.0
36081 775.0
36087 1715.0
37083 

In [173]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 1867
36061 4282
36047 8521 8521
36081 8197
36005 5795


In [174]:
print (covid_cases_county_ny_times)

{1001.0: 219.0, 1003.0: 676.0, 1005.0: 65.0, 1007.0: 58.0, 1009.0: 133.0, 1011.0: 30.0, 1013.0: 59.0, 1015.0: 598.0, 1017.0: 80.0, 1019.0: 25.0, 1021.0: 148.0, 1023.0: 5.0, 1025.0: 53.0, 1027.0: 44.0, 1029.0: 34.0, 1031.0: 95.0, 1033.0: 65.0, 1035.0: 16.0, 1037.0: 49.0, 1039.0: 142.0, 1041.0: 49.0, 1043.0: 88.0, 1045.0: 141.0, 1047.0: 123.0, 1049.0: 93.0, 1051.0: 277.0, 1053.0: 82.0, 1055.0: 229.0, 1057.0: 58.0, 1059.0: 37.0, 1061.0: 47.0, 1063.0: 29.0, 1065.0: 72.0, 1067.0: 51.0, 1069.0: 206.0, 1071.0: 81.0, 1073.0: 1376.0, 1075.0: 43.0, 1077.0: 139.0, 1079.0: 51.0, 1081.0: 376.0, 1083.0: 216.0, 1085.0: 24.0, 1087.0: 46.0, 1089.0: 943.0, 1091.0: 186.0, 1093.0: 111.0, 1095.0: 158.0, 1097.0: 1249.0, 1099.0: 20.0, 1101.0: 604.0, 1103.0: 253.0, 1105.0: 15.0, 1107.0: 69.0, 1109.0: 55.0, 1111.0: 60.0, 1113.0: 101.0, 1115.0: 228.0, 1117.0: 656.0, 1119.0: 20.0, 1121.0: 243.0, 1123.0: 175.0, 1125.0: 630.0, 1127.0: 157.0, 1129.0: 48.0, 1131.0: 19.0, 1133.0: 40.0, 2013.0: 9.0, 2016.0: 157.0, 202

In [175]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [176]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [177]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [178]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [179]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [180]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [181]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [182]:
#ACI
ACI_State = total_state(ACI_total)

In [183]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [184]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [185]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [186]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [187]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [188]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [189]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 64.0 38921 164.43565170473522
1001 89.0 55869 159.30122250264012
1009 125.0 57826 216.16573859509566
1013 65.0 19448 334.22459893048125
1015 285.0 113605 250.8692399102152
1017 108.0 33254 324.7729596439526
1031 102.0 52342 194.87218677161744
1033 118.0 55241 213.609456744085
1039 106.0 37049 286.1075872493185
1043 179.0 83768 213.68541686562887
1045 106.0 49172 215.5698364923127
1051 183.0 81209 225.34448152298393
1055 319.0 102268 311.92552900222944
1067 41.0 17205 238.30281894798023
1069 259.0 105882 244.6119264841994
1071 102.0 51626 197.57486537791036
1077 210.0 92729 226.46636974409302
1079 84.0 32924 255.13303365326203
1083 134.0 98915 135.46984784916341
1089 442.0 372909 118.52757643285626
1095 206.0 96774 212.86709240085148
1097 721.0 413210 174.4875487040488
1103 249.0 119679 208.05655127465974
1111 40.0 22722 176.04084147522227
1113 31.0 57961 53.484239402356756
1115 220.0 89512 245.77710251139513
1117 215.0 217702 98.75885384608318
1121 163.0 79978 203.80604666283227


27107 8.0 6375 125.49019607843138
27111 70.0 58746 119.15704899056958
27115 16.0 29579 54.09243044051522
27121 5.0 11249 44.44839541292559
27125 4.0 4055 98.64364981504315
27127 27.0 15170 177.98286090969017
27129 40.0 14548 274.95188342040143
27133 13.0 9315 139.55984970477724
27135 17.0 15165 112.10023079459282
27137 262.0 199070 131.61199578037875
27143 10.0 14865 67.27211570803902
27147 11.0 36649 30.014461513274576
27151 18.0 9266 194.25857975393913
27153 30.0 24664 121.63477132662992
27157 3.0 21627 13.871549452073797
27161 17.0 18612 91.33892112615517
27163 253.0 262440 96.40298734948941
27167 10.0 6207 161.10842597067827
27173 17.0 9709 175.09527242764443
28003 60.0 36953 162.36841393120991
28033 229.0 184945 123.8205953121198
28045 73.0 47632 153.25831373866308
28057 71.0 23390 303.5485250106883
28059 217.0 143617 151.09631868093612
28081 160.0 85436 187.2746851444356
28089 195.0 106272 183.4914182475158
28093 92.0 35294 260.66753555845185
28095 126.0 35252 357.4265289912629
2

54029 86.0 28810 298.5074626865672
54031 20.0 13776 145.18002322880372
54033 70.0 67256 104.07993338884262
54035 50.0 28576 174.97200447928333
54037 37.0 57146 64.74643894585797
54041 16.0 15907 100.58464826805809
54049 33.0 56072 58.85290340990156
54051 66.0 30531 216.17372506632606
54053 36.0 26516 135.76708402473977
54057 77.0 26868 286.5862736340628
54065 13.0 17884 72.6906732274659
54067 12.0 24496 48.98758981058132
54073 17.0 7460 227.88203753351206
54079 74.0 56450 131.08945969884851
54083 10.0 28695 34.84927687750479
54085 9.0 9554 94.20138162026376
54091 21.0 16695 125.78616352201257
54095 4.0 8591 46.560353858689325
54097 30.0 24176 124.0900066181337
54103 19.0 15065 126.12014603385332
54107 130.0 83518 155.65506836849542
55003 16.0 15562 102.81454825857858
55005 76.0 45244 167.97807444080985
55009 249.0 264542 94.12494046314006
55011 7.0 13031 53.71805694114036
55015 45.0 50089 89.84008464932421
55017 94.0 64658 145.38030870116614
55021 60.0 57532 104.28978655357018
55023 17

17085 25.0 21235 117.73016246762421
17087 18.0 12417 144.9625513409036
17089 763.0 532403 143.3124907260102
17093 93.0 128990 72.09861229552679
17095 162.0 49699 325.9622930038834
17101 33.0 15678 210.48603138155377
17103 82.0 34096 240.49741905208822
17107 66.0 28618 230.62408274512543
17113 192.0 171517 111.94225645271314
17121 117.0 37205 314.4738610401828
17135 73.0 28414 256.9156049834589
17137 98.0 33658 291.16406203577156
17139 30.0 14501 206.8822839804151
17145 66.0 20916 315.5479059093517
17157 84.0 31782 264.3005474797055
17169 17.0 6768 251.18203309692672
17177 94.0 44498 211.2454492336734
17183 131.0 75758 172.91903165342273
18011 93.0 67843 137.08120218740328
18017 101.0 37689 267.98270052269896
18039 429.0 206341 207.9082683519029
18053 164.0 65769 249.35760008514652
18057 412.0 338011 121.88952430542201
18075 30.0 20436 146.7997651203758
18077 73.0 32308 225.95022904543768
18121 15.0 16937 88.56350002952117
18123 36.0 19169 187.80322395534455
18133 59.0 37576 157.0151160

46099 329.0 193134 170.34804850518293
46103 184.0 113775 161.72269830806417
46127 39.0 15932 244.79035902585989
46135 28.0 22814 122.73165600070132
47005 39.0 16160 241.33663366336634
47017 81.0 27767 291.7131847156697
47023 47.0 17297 271.7234202462855
47045 101.0 37159 271.8049463117953
47059 145.0 69069 209.9349925436882
47073 94.0 56786 165.53375832071285
47085 21.0 18582 113.01259283177268
47093 573.0 470313 121.83375751892889
47105 65.0 54068 120.2189835022564
47109 52.0 25694 202.3818790379077
47123 90.0 46545 193.36126329358686
47125 209.0 208993 100.00334939447733
47153 25.0 15026 166.37827765206976
47169 22.0 11284 194.96632399858206
47187 202.0 238412 84.7272788282469
48007 34.0 23510 144.6193109315185
48035 30.0 18685 160.55659620016056
48037 179.0 93245 191.96739771569523
48045 4.0 1546 258.732212160414
48071 42.0 43837 95.80947601341333
48093 50.0 13635 366.70333700036673
48121 634.0 887207 71.46021165297388
48125 9.0 2211 407.05563093622794
48129 13.0 3278 396.5832824893

48225 41.0 22968 178.50923023336816
48245 340.0 251565 135.15393635839644
48253 49.0 20083 243.98745207389337
48263 1.0 762 131.23359580052494
48311 4.0 743 538.3580080753701
48315 27.0 9854 274.0004059265273
48317 18.0 5771 311.9043493328713
48327 7.0 2138 327.4087932647334
48339 448.0 607391 73.75808992889259
48383 8.0 3849 207.84619381657575
48385 12.0 3452 347.62456546929315
48391 17.0 6948 244.6747265400115
48403 38.0 10542 360.4629102637071
48405 29.0 8237 352.0699283719801
48411 19.0 6055 313.79025598678777
48415 61.0 16703 365.2038555948033
48421 12.0 3022 397.08802117802776
48425 11.0 9128 120.50832602979841
48455 21.0 14651 143.3349259436216
48461 9.0 3657 246.1033634126333
48475 20.0 11998 166.6944490748458
48481 101.0 41556 243.0455289248243
49031 2.0 1479 135.2265043948614
51121 74.0 98535 75.10021819657989
51520 27.0 16762 161.0786302350555
51540 41.0 47266 86.74311344306689
51678 20.0 7446 268.60059092130007
51680 109.0 82168 132.6550481939441
51750 11.0 18249 60.2772754

48013 134.0 51153 261.95922037808145
48017 17.0 7000 242.85714285714283
48025 64.0 32565 196.5300168892983
48069 29.0 7530 385.1261620185923
48079 13.0 2853 455.66070802663864
48095 7.0 2726 256.78650036683786
48101 7.0 1398 500.7153075822604
48103 11.0 4797 229.30998540754638
48113 3356.0 2635516 127.3374929235869
48117 63.0 18546 339.6958912973148
48153 29.0 5712 507.703081232493
48169 19.0 6229 305.024883608926
48177 50.0 20837 239.9577674329318
48195 19.0 5399 351.9170216706798
48201 4980.0 4713325 105.65789543475147
48207 22.0 5658 388.8299752562743
48249 82.0 40482 202.55916209673438
48255 31.0 15601 198.70521120440998
48271 6.0 3667 163.6214889555495
48279 79.0 12893 612.7355929574188
48301 0.0 169 0.0
48341 68.0 20940 324.7373447946514
48357 25.0 9836 254.16836112240748
48369 34.0 9605 353.98230088495575
48371 32.0 15823 202.23724957340582
48375 403.0 117415 343.2270152876549
48389 39.0 15976 244.11617426139208
48419 54.0 25274 213.6583049774472
48435 8.0 3776 211.864406779661


In [190]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1917.0, 854757, 224.27426742337295)

In [191]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6203.091331543696,
 'ALABAMA': 16413.362702989725,
 'ALASKA': 600.6489956338517,
 'ARIZONA': 4208.070152617309,
 'ARKANSAS': 15895.795418574007,
 'CALIFORNIA': 5057.419068481651,
 'COLORADO': 6982.574364808959,
 'CONNECTICUT': 1474.8953254530745,
 'DELAWARE': 467.7645122204999,
 'FLORIDA': 11145.819400221108,
 'GEORGIA': 36899.13387373144,
 'IDAHO': 4445.24984092033,
 'ILLINOIS': 21409.982497648332,
 'INDIANA': 18095.03551300566,
 'IOWA': 22545.047882776864,
 'KANSAS': 16874.545972862477,
 'KENTUCKY': 14633.832640613151,
 'LOUISIANA': 15570.783304673389,
 'MAINE': 703.2188784657961,
 'MARYLAND': 3121.762668778784,
 'MASSACHUSETTS': 2665.945752533137,
 'MICHIGAN': 12561.64938095943,
 'MINNESOTA': 10913.370956973737,
 'MISSISSIPPI': 22091.792421464328,
 'MISSOURI': 15449.508781430752,
 'MONTANA': 10053.441244245823,
 'NEBRASKA': 12646.765549447637,
 'NEVADA': 1932.896212676431,
 'NEW HAMPSHIRE': 701.1928399615758,
 'NEW JERSEY': 5051.462411439386,
 'NEW YORK': 8338.1780368

In [192]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [193]:
Covid_death_capita[72049]

nan

In [194]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [195]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [196]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [197]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76277412.0 1000000 10122.288889402804
ALABAMA 957116.0000000001 76277412.0 1000000 12547.830018144821
ALASKA 231145.0 76277412.0 1000000 3030.3204309029256
ARIZONA 1839932.0 76277412.0 1000000 24121.583988717393
ARKANSAS 830467.0000000003 76277412.0 1000000 10887.456433367199
CALIFORNIA 11847711.0 76277412.0 1000000 155323.9771690209
COLORADO 1337805.0 76277412.0 1000000 17538.67842291241
CONNECTICUT 874974.0 76277412.0 1000000 11470.945028916292
DELAWARE 239009.0 76277412.0 1000000 3133.417793461582
FLORIDA 3892551.999999999 76277412.0 1000000 51031.516381284666
GEORGIA 1928702.999999999 76277412.0 1000000 25285.37543984842
IDAHO 340742.0000000001 76277412.0 1000000 4467.141596256571
ILLINOIS 2987496.0000000005 76277412.0 1000000 39166.195098491284
INDIANA 1602975.9999999995 76277412.0 1000000 21015.08110946396
IOWA 699741.0 76277412.0 1000000 9173.633211362756
KANSAS 401103.0000000001 76277412.0 1000000 5258.47678209114
KENTUCKY 1416013.0000000002 76277412.0 10000

In [198]:
print (Covid_state)

{'NEW MEXICO': 4359.0, 'ALABAMA': 12568.0, 'ALASKA': 1724.0, 'ARIZONA': 19718.0, 'ARKANSAS': 6614.0, 'CALIFORNIA': 84375.0, 'COLORADO': 13367.0, 'CONNECTICUT': 11756.0, 'DELAWARE': 3530.0, 'FLORIDA': 78040.0, 'GEORGIA': 38497.0, 'IDAHO': 3393.0, 'ILLINOIS': 23223.0, 'INDIANA': 12784.0, 'IOWA': 3438.0, 'KANSAS': 6891.0, 'KENTUCKY': 15079.0, 'LOUISIANA': 9443.0, 'MAINE': 1862.0, 'MARYLAND': 10492.0, 'MASSACHUSETTS': 20137.0, 'MICHIGAN': 15269.0, 'MINNESOTA': 9958.0, 'MISSISSIPPI': 7178.0, 'MISSOURI': 7737.0, 'MONTANA': 2434.0, 'NEBRASKA': 4126.0, 'NEVADA': 5600.0, 'NEW HAMPSHIRE': 4424.0, 'NEW JERSEY': 39024.0, 'NEW YORK': 73146.0, 'NORTH CAROLINA': 38349.0, 'NORTH DAKOTA': 1170.0, 'OHIO': 27706.0, 'OKLAHOMA': 11368.0, 'OREGON': 5007.0, 'PENNSYLVANIA': 34704.0, 'RHODE ISLAND': 4574.0, 'SOUTH CAROLINA': 31389.0, 'SOUTH DAKOTA': 1918.0, 'TENNESSEE': 20139.0, 'TEXAS': 78230.0, 'UTAH': 9612.0, 'VERMONT': 1426.0, 'VIRGINIA': 27433.0, 'WASHINGTON': 10596.0, 'WEST VIRGINIA': 3986.0, 'WISCONSIN'

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [199]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [200]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [201]:
Total_covid_death_cap


{'NEW MEXICO': 6203.091331543696,
 'ALABAMA': 16413.362702989725,
 'ALASKA': 600.6489956338517,
 'ARIZONA': 4208.070152617309,
 'ARKANSAS': 15895.795418574007,
 'CALIFORNIA': 5057.419068481651,
 'COLORADO': 6982.574364808959,
 'CONNECTICUT': 1474.8953254530745,
 'DELAWARE': 467.7645122204999,
 'FLORIDA': 11145.819400221108,
 'GEORGIA': 36899.13387373144,
 'IDAHO': 4445.24984092033,
 'ILLINOIS': 21409.982497648332,
 'INDIANA': 18095.03551300566,
 'IOWA': 22545.047882776864,
 'KANSAS': 16874.545972862477,
 'KENTUCKY': 14633.832640613151,
 'LOUISIANA': 15570.783304673389,
 'MAINE': 703.2188784657961,
 'MARYLAND': 3121.762668778784,
 'MASSACHUSETTS': 2665.945752533137,
 'MICHIGAN': 12561.64938095943,
 'MINNESOTA': 10913.370956973737,
 'MISSISSIPPI': 22091.792421464328,
 'MISSOURI': 15449.508781430752,
 'MONTANA': 10053.441244245823,
 'NEBRASKA': 12646.765549447637,
 'NEVADA': 1932.896212676431,
 'NEW HAMPSHIRE': 701.1928399615758,
 'NEW JERSEY': 5051.462411439386,
 'NEW YORK': 8338.1780368

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [202]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [203]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [204]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [205]:
#print (Medicaid_demand)

In [206]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [207]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [208]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [209]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)